In [1]:
sc

In [2]:
import csv
import time
import datetime as dt
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 
from matplotlib import dates
import dateutil.parser
import seaborn as sns
from operator import itemgetter
from pyspark.mllib.fpm import FPGrowth,PrefixSpan,PrefixSpanModel
from functools import reduce
from itertools import groupby




ModuleNotFoundError: No module named 'numpy'

In [4]:
staypoint1 = "/Users/indrikwijaya/Documents/DataSpark_Internship/4th_Project_Staypoint/part-00566"
staypoint = "/Users/indrikwijaya/Desktop/part-00414"
grid_name = "/Users/indrikwijaya/Documents/grid_name.csv"
mcc = "/Users/indrikwijaya/Documents/DataSpark_Internship/4th_Project_Staypoint/mcc_list.txt"
grid_submtz = "/Users/indrikwijaya/Documents/grid_sub-mtz.csv"

def round_time(time):
    time = datetime.strptime(time, "%H:%M:%S")
    if int(time.minute*60+time.second)//1800==1:
        return str(int(time.hour+1))
    else:
        return str(int(time.hour))
    
m = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun", "Ave"]

mcc = sc.textFile(mcc).map(lambda x: x.strip().replace("|",",").split(",")) \
        .map(lambda x: (x[0],x[3])) \
        .distinct()
        
grid_name = sc.textFile(grid_name).map(lambda x: (x.split(",")[0], x.split(",")[1]))

grid_submtz = sc.textFile(grid_submtz).map(lambda x: (x.split(",")[0],x.split(",")[1]))

staypoint = sc.textFile(staypoint1).map(lambda x: x.strip().replace("\n", "").split(";")) \
            .filter(lambda x: x[0][:3] != '525') \
            .flatMap(lambda x: tuple((i.split(",")[0][5:],x[0]+","+\
                                      datetime.strptime(i.split(",")[1].split(" ")[0],'%Y-%m-%d').strftime('%d-%a')+" "+\
                                      round_time(i.split(",")[1].split(" ")[1])) for i in x[1:]))
                                    #+" "+\
                                    #  datetime.strptime(i.split(",")[2].split(" ")[0],'%Y-%m-%d').strftime('%d-%a')+" "+\
                                    #  round_time(i.split(",")[2].split(" ")[1]))\
                                    # for i in x[1:])) 
        
staypoint_name = grid_submtz.join(staypoint) \
                        .map(lambda x: x[1]) \
                        .map(lambda x: (x[1].split(",")[0],[(x[1].split(",")[1].split(" ")[0],\
                        int(x[1].split(",")[1].split(" ")[1]), x[0])]))\
                        .reduceByKey(lambda x,y : x +y)\
                        .map(lambda x: (x[0][:3],x[1]))\
                        .filter(lambda x: len(x[1])>=3) \

                        #,x[1].split(",")[1].split(" ")[2],int(x[1].split(",")[1].split(" ")[3]) \
                        #,x[0])]))\
                        
#.mapValues(lambda x: tuple(i[4] for i in x)) \                    


stayname_ctry = mcc.join(staypoint_name) \
                .map(lambda x: (x[1][0], x[1][1])) \
                .mapValues(lambda x: sorted(x, key = lambda x: x[0].split("-")[0]))\
                .mapValues(lambda x: sorted(x, key = itemgetter(1))) \
                .mapValues(lambda x: list((i[0], str(i[1])+" "+i[2]) for i in x))\
                .mapValues(lambda x: [(key,)+tuple(elem[1:] for elem in group) for key, group in groupby(x, lambda y:y[0])])

stayname_ctry.take(14)

[('Australia',
  [('19-Mon',
    ('0 453',),
    ('10 519',),
    ('11 519',),
    ('14 559',),
    ('16 519',),
    ('18 544',),
    ('19 519',),
    ('22 705',),
    ('22 453',))]),
 ('Australia', [('19-Mon', ('0 461',), ('7 461',), ('13 510',))]),
 ('Netherlands', [('19-Mon', ('0 661',), ('10 553',), ('20 1071',))]),
 ('Netherlands', [('19-Mon', ('0 1060',), ('12 1196',), ('18 1060',))]),
 ('Netherlands',
  [('19-Mon', ('0 1351',), ('7 1414',), ('17 664',), ('23 1351',))]),
 ('Netherlands',
  [('19-Mon', ('0 990',), ('10 1061',), ('15 991',), ('20 990',))]),
 ('Philippines',
  [('19-Mon', ('1 704',), ('2 864',), ('3 705',), ('8 392',), ('20 705',))]),
 ('Philippines', [('19-Mon', ('1 1356',), ('11 1071',), ('23 1356',))]),
 ('Philippines',
  [('19-Mon', ('2 597',), ('9 452',), ('18 531',), ('22 657',), ('22 597',))]),
 ('Philippines', [('19-Mon', ('2 457',), ('13 673',), ('16 457',))])]

In [8]:
pattern1 = "/Users/indrikwijaya/Desktop/June/20170601"
pattern2 = "/Users/indrikwijaya/Desktop/June/20170608"
pattern3 = "/Users/indrikwijaya/Desktop/June/20170615"
pattern4 = "/Users/indrikwijaya/Desktop/June/20170622"
filenames = [pattern1, pattern2, pattern3, pattern4]
with open('/Users/indrikwijaya/Desktop/June/June', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [21]:
pattern = "/Users/indrikwijaya/Desktop/June/total"
pattern = sc.textFile(pattern).map(lambda x: eval(x))    
pattern = mcc.join(pattern) \
                .map(lambda x: (x[1][0], 1)) \
                .reduceByKey(lambda x,y : x+y) \
                .sortBy(lambda x: x[1], ascending= False)
pattern.collect()

[('Malaysia', 806999),
 ('Indonesia', 331489),
 ('China', 176184),
 ('India', 124656),
 ('United States', 77948),
 ('Guam', 77947),
 ('Netherlands', 66822),
 ('Australia', 60676),
 ('United Kingdom', 59120),
 ('Japan', 48516),
 ('Korea S', 45108),
 ('Philippines', 42189),
 ('Thailand', 38779),
 ('Hongkong', 37286),
 ('Viet Nam', 32894),
 ('Germany', 21173),
 ('Burma', 17753),
 ('Italy', 17166),
 ('France', 15597),
 ('New Zealand', 13729),
 ('United Arab Emirates', 13311),
 ('Taiwan', 11504),
 ('Switzerland', 11239),
 ('Canada', 10314),
 ('Sweden', 8965),
 ('Russian Federation', 6111),
 ('Austria', 6051),
 ('Brunei', 5782),
 ('Belgium', 4076),
 ('Spain', 3828),
 ('Cambodia', 3779),
 ('Poland', 2566),
 ('Denmark', 2287),
 ('Saudi Arabia', 1979),
 ('Greece', 1856),
 ('Qatar', 1693),
 ('Turkey', 1669),
 ('Macao', 1630),
 ('Ireland', 1363),
 ('Norway', 1360),
 ('Finland', 1337),
 ('South Africa', 1289),
 ('Estonia', 1236),
 ('Romania', 965),
 ('Ukraine', 820),
 ('Portugal', 793),
 ('Czech R

In [32]:
def filter_by_nationality(pattern, country):
    pattern = sc.textFile(pattern).map(lambda x: eval(x))    
    pattern = mcc.join(pattern) \
                .map(lambda x: (x[1][0], x[1][1])) \
                .filter(lambda x: x[0]==country)\
                .mapValues(lambda x: sorted(x, key = itemgetter(1))) \
                .mapValues(lambda x: sorted(x, key = lambda x: x[0].split("-")[0]))\
                .mapValues(lambda x: list((i[0], i[2]) for i in x))\
                .mapValues(lambda x: [(key,list(elem[1] for elem in group)) for key, group in groupby(x, lambda y:y[0])])\
                .flatMap(lambda x: x[1])\
                .map(lambda x: x[1])\
                .map(lambda x: list(set(x)))\
                .filter(lambda x: len(x) >=4)\
                .cache()\
                #.coalesce(1).saveAsTextFile("/Users/indrikwijaya/Desktop/"+country)
    return pattern

In [42]:
country = ["China", "Indonesia", "India"]
data = "/Users/indrikwijaya/Desktop/june/total"
def FPG_analysis(data, countries,minSupport):
    result = {}
    for country in countries:
        pattern = filter_by_nationality(data, country)
        print(pattern.count())
        model = FPGrowth.train(pattern,minSupport, numPartitions = 5)
        result[country]=sorted(model.freqItemsets().collect())
    return result

FPG_analysis(data, country, 0.05)

89316
89239
53476


{'China': [FreqItemset(items=['1078'], freq=21753),
  FreqItemset(items=['1078', '531'], freq=6695),
  FreqItemset(items=['1119'], freq=9241),
  FreqItemset(items=['1483'], freq=8050),
  FreqItemset(items=['386'], freq=6197),
  FreqItemset(items=['452'], freq=6349),
  FreqItemset(items=['454'], freq=8795),
  FreqItemset(items=['466'], freq=4766),
  FreqItemset(items=['467'], freq=11495),
  FreqItemset(items=['467', '531'], freq=4827),
  FreqItemset(items=['515'], freq=7129),
  FreqItemset(items=['519'], freq=13090),
  FreqItemset(items=['525'], freq=8120),
  FreqItemset(items=['531'], freq=21944),
  FreqItemset(items=['545'], freq=8061),
  FreqItemset(items=['546'], freq=6218),
  FreqItemset(items=['566'], freq=7080),
  FreqItemset(items=['651'], freq=5532),
  FreqItemset(items=['657'], freq=7101)],
 'India': [FreqItemset(items=['1078'], freq=8935),
  FreqItemset(items=['1386'], freq=4614),
  FreqItemset(items=['1485'], freq=5863),
  FreqItemset(items=['386'], freq=2862),
  FreqItemset

In [26]:
total= "/Users/indrikwijaya/Desktop/June/total"

indo = filter_by_nationality(total, "Indonesia")
india = filter_by_nationality(total, "India")
china = filter_by_nationality(total, "China")

In [34]:
def convert_text(country):
    file = "/Users/indrikwijaya/Desktop/country/"+country
    tourist = open(file, 'r')
    converted = list(map(lambda x: eval(x),[row for row in tourist]))
    return converted

indo = convert_text("Indonesia")
china = convert_text("China")
india = convert_text("India")

In [45]:
from pymining import itemmining, assocrules,perftesting
import pprint

def relim(country, min_support, min_confidence):
    tourist = convert_text(country)
    relim_input = itemmining.get_relim_input(tourist)
    relim_report = itemmining.relim(relim_input, min_support)
    relim_rules = assocrules.mine_assoc_rules(relim_report, min_support, min_confidence)
    print(country)
    pprint.pprint(relim_report)
    print(*sorted(relim_rules), sep ="\n")
    return [relim_report, relim_rules]

def sam(country, min_support, min_confidence):
    tourist = convert_text(country)
    sam_input = itemmining.get_sam_input(tourist)
    sam_report = itemmining.sam(sam_input, min_support)
    sam_rules = assocrules.mine_assoc_rules(sam_report, min_support, min_confidence)
    print(country)
    pprint.pprint(sam_report)
    print(*sam_rules, sep = "\n")
    return [sam_report, sam_rules]

In [ ]:
a = relim("Indonesia", 1500, 0.2)


In [53]:
list(a[0].keys())

[frozenset({'751'}),
 frozenset({'544'}),
 frozenset({'598'}),
 frozenset({'406'}),
 frozenset({'569'}),
 frozenset({'602'}),
 frozenset({'477'}),
 frozenset({'462'}),
 frozenset({'716'}),
 frozenset({'432'}),
 frozenset({'557'}),
 frozenset({'582'}),
 frozenset({'451'}),
 frozenset({'1485'}),
 frozenset({'632'}),
 frozenset({'625'}),
 frozenset({'512'}),
 frozenset({'518'}),
 frozenset({'571'}),
 frozenset({'473'}),
 frozenset({'599'}),
 frozenset({'722'}),
 frozenset({'562'}),
 frozenset({'1483'}),
 frozenset({'1531'}),
 frozenset({'563'}),
 frozenset({'464'}),
 frozenset({'600'}),
 frozenset({'415'}),
 frozenset({'734'}),
 frozenset({'664'}),
 frozenset({'546'}),
 frozenset({'657'}),
 frozenset({'465'}),
 frozenset({'434'}),
 frozenset({'564'}),
 frozenset({'466'}),
 frozenset({'719'}),
 frozenset({'750'}),
 frozenset({'581'}),
 frozenset({'580'}),
 frozenset({'597'}),
 frozenset({'545'}),
 frozenset({'408'}),
 frozenset({'467'}),
 frozenset({'717'}),
 frozenset({'525'}),
 frozenset

In [41]:
relim("China", 2500, 0.2)

China
{frozenset({'584'}): 2514,
 frozenset({'563'}): 2564,
 frozenset({'415'}): 2629,
 frozenset({'719'}): 2646,
 frozenset({'508'}): 2652,
 frozenset({'389'}): 2698,
 frozenset({'569'}): 2736,
 frozenset({'453'}): 2741,
 frozenset({'652'}): 3110,
 frozenset({'1531'}): 3120,
 frozenset({'580'}): 3134,
 frozenset({'1386'}): 3152,
 frozenset({'543'}): 3203,
 frozenset({'565'}): 3355,
 frozenset({'424'}): 3463,
 frozenset({'477'}): 3468,
 frozenset({'462'}): 3622,
 frozenset({'408'}): 3831,
 frozenset({'395'}): 4202,
 frozenset({'625'}): 4275,
 frozenset({'465'}): 4284,
 frozenset({'464'}): 4400,
 frozenset({'466'}): 4766,
 frozenset({'651'}): 5532,
 frozenset({'386'}): 6197,
 frozenset({'386', '531'}): 4343,
 frozenset({'546'}): 6218,
 frozenset({'452'}): 6349,
 frozenset({'452', '531'}): 4408,
 frozenset({'566'}): 7080,
 frozenset({'657'}): 7101,
 frozenset({'657', '1078'}): 2522,
 frozenset({'657', '531'}): 2968,
 frozenset({'515'}): 7129,
 frozenset({'1483'}): 8050,
 frozenset({'1483